# Collaboratory Filtering (Movie Recommendation Case)

### Zumarsiyah Mahsyari
##### Astra Data Scientist Bootcamp

Collaboratory Filtering adalah salah satu algoritma dalam Recommendation System. Algoritma ini menyaring informasi dengan berdasarkan pengguna lainnya yang memiliki similarity atau kesamaan. Pada Practice Case kali ini kita akan menemukan rekomendasi film bagi penggunanya atau user. <br>
Langkah-langkah untuk mendapatkan rekomendasi film berdasarkan algoritma ini adalah: <br>
1. Pilih seorang user dengan film-film yang telah ditonton
2. Berdasarkan rating yang diberikan olehnya, temukan user lain yang similar dengannya
3. Mendapaykan film-film yang telah ditonton oleh tiap user lain.
4. Hitung similarity score
5. Rekomendasikan film dengan score tertinggi.

Data yang digunakan pada Practice Case kali ini adalah dataset movies dan ratings. Data movies berisi daftar kumpulan film yang terdiri dari 3 feature, yaitu movieId, title (judul film), dan genres film. Sedangkan data ratings berisi kumpulan rating yang diberikan oleh user untuk beberapa film. Data ini terdiri dari userId, movieId, rating film, dan timestamp.

In [1]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Load & Read Data

In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('C:/Users/zumars064124/Documents/ADSB 2/Tugas MBA/moviedataset/movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('C:/Users/zumars064124/Documents/ADSB 2/Tugas MBA/moviedataset/ratings.csv')

In [3]:
movies_df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

### Preprocessing Data

Kita dapat menghapus tahun film dari judulnya dan membuat kolom 'tahun' baru.

In [4]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [5]:
movies_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995

In [6]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

In [7]:
movies_df.head()

movieId                        title  year
0        1                    Toy Story  1995
1        2                      Jumanji  1995
2        3             Grumpier Old Men  1995
3        4            Waiting to Exhale  1995
4        5  Father of the Bride Part II  1995

In [8]:
ratings_df.head()

userId  movieId  rating   timestamp
0       1      169     2.5  1204927694
1       1     2471     3.0  1204927438
2       1    48516     5.0  1204927435
3       2     2571     3.5  1436165433
4       2   109487     4.0  1436165496

Setiap baris dalam pada data 'ratings' memiliki id pengguna yang terkait dengan setidaknya satu film serta rating yang mereka berikan. 'Timestamp' adalah waktu ketika mereka memberi ratings tersebut. Kita tidak akan memerlukan feature 'Timestamp', jadi mari kita hapus.

In [9]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

In [10]:
ratings_df.head()

userId  movieId  rating
0       1      169     2.5
1       1     2471     3.0
2       1    48516     5.0
3       2     2571     3.5
4       2   109487     4.0

### Collaborative Filtering

In [11]:
#creating an input user to recommend movies
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

rating                title
0     5.0  Breakfast Club, The
1     3.5            Toy Story
2     2.0              Jumanji
3     5.0         Pulp Fiction
4     4.5                Akira

Kita butuh menambahkan movieId pada data inputMovies agar dapat dianalisis lebih lanjut.

In [12]:
#Add movieId to input user

#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0

Kita dapat menyaring user lain yang menonton film yang sama.

In [14]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

userId  movieId  rating
19        4      296     4.0
441      12     1968     3.0
479      13        2     2.0
531      13     1274     5.0
681      14      296     2.0

Dapat dilihat bahwa user dengan userId 4, 12, 13, 13, dan 14 menonton film yang sama dengan input user.

In [17]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

In [18]:
#look at one of the users, e.g. the one with userID=1130u
serSubsetGroup.get_group(1130)

userId  movieId  rating
104167    1130        1     0.5
104168    1130        2     4.0
104214    1130      296     4.0
104363    1130     1274     4.5
104443    1130     1968     4.5

Urutkan user berdasarkan film-film yang memiliki kesamaan paling banyak dengan input user agar memiliki prioritas yang tinggi. Ini memberikan rekomendasi yang lebih kuat.

In [19]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [20]:
userSubsetGroup[0:3]

[(75,       userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0), (106,       userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5), (686,        userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

##### Similarity of users to input user <br>

Kita akan membandingkan seluruh user lainnya untuk menemukan user yang paling similar dengan input user menggunakan Pearson Correlation. Pearson Correlation digunakan untuk menghitung seberapa kuatnya hubungan linear antara 2 variabel. Kita akan hanya mengambil 100 user untuk menghemat waktu.

In [21]:
userSubsetGroup = userSubsetGroup[0:100]

In [22]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [23]:
pearsonCorrelationDict.items()

dict_items([(75, 0.8272781516947562), (106, 0.5860090386731182), (686, 0.8320502943378437), (815, 0.5765566601970551), (1040, 0.9434563530497265), (1130, 0.2891574659831201), (1502, 0.8770580193070299), (1599, 0.4385290096535153), (1625, 0.716114874039432), (1950, 0.179028718509858), (2065, 0.4385290096535153), (2128, 0.5860090386731196), (2432, 0.1386750490563073), (2791, 0.8770580193070299), (2839, 0.8204126541423674), (2948, -0.11720180773462392), (3025, 0.45124262819713973), (3040, 0.89514359254929), (3186, 0.6784622064861935), (3271, 0.26989594817970664), (3429, 0.0), (3734, -0.15041420939904673), (4099, 0.05860090386731196), (4208, 0.29417420270727607), (4282, -0.4385290096535115), (4292, 0.6564386345361464), (4415, -0.11183835382312353), (4586, -0.9024852563942795), (4725, -0.08006407690254357), (4818, 0.4885967564883424), (5104, 0.7674257668936507), (5165, -0.4385290096535153), (5547, 0.17200522903844556), (6082, -0.04728779924109591), (6207, 0.9615384615384616), (6366, 0.65779

In [24]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

similarityIndex  userId
0         0.827278      75
1         0.586009     106
2         0.832050     686
3         0.576557     815
4         0.943456    1040

#### The top 50 similar users to input user

In [25]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

similarityIndex  userId
64         0.961678   12325
34         0.961538    6207
55         0.961538   10707
67         0.960769   13053
4          0.943456    1040

Berdasarkan tabel di atas, user dengan userId 12325 adalah user yang paling similar dengan inpt user karena emmiliki similarityindex yang paling tinggi, yaitu sebesar 0,9617

##### Rating of selected users to all movies
Kita akan melakukan ini dengan mengambil weighted average dari ratings film menggunakan Pearson Correlation sebagai bobot. Tetapi untuk melakukan ini, pertama-tama kita perlu membuat film yang ditonton oleh pengguna di __pearsonDF__ dari DataFrame kemudian menyimpan korelasinya dalam kolom baru yang disebut __similarityIndex__ dengan menggabungkan dua tabel ini.

In [26]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

similarityIndex  userId  movieId  rating
0         0.961678   12325        1     3.5
1         0.961678   12325        2     1.5
2         0.961678   12325        3     3.0
3         0.961678   12325        5     0.5
4         0.961678   12325        6     2.5

Sekarang yang perlu kita lakukan hanyalah mengalikan peringkat film berdasarkan bobotnya (The similarity index), lalu menjumlahkan peringkat baru dan membaginya dengan jumlah bobot.

Kita dapat dengan mudah melakukan ini dengan hanya mengalikan dua kolom, lalu mengelompokkan DataFrame dengan movieId dan kemudian membagi dua kolom.

In [27]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

similarityIndex  userId  movieId  rating  weightedRating
0         0.961678   12325        1     3.5        3.365874
1         0.961678   12325        2     1.5        1.442517
2         0.961678   12325        3     3.0        2.885035
3         0.961678   12325        5     0.5        0.480839
4         0.961678   12325        6     2.5        2.404196

In [28]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

sum_similarityIndex  sum_weightedRating
movieId                                         
1                  38.376281          140.800834
2                  38.376281           96.656745
3                  10.253981           27.254477
4                   0.929294            2.787882
5                  11.723262           27.151751

In [29]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

weighted average recommendation score  movieId
movieId                                                
1                                     3.668955        1
2                                     2.518658        2
3                                     2.657941        3
4                                     3.000000        4
5                                     2.316058        5

In [30]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

weighted average recommendation score  movieId
movieId                                                
5073                                       5.0     5073
3329                                       5.0     3329
2284                                       5.0     2284
26801                                      5.0    26801
6776                                       5.0     6776
6672                                       5.0     6672
3759                                       5.0     3759
3769                                       5.0     3769
3775                                       5.0     3775
90531                                      5.0    90531

##### Movies Recommendation

In [31]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

movieId                                    title  year
2200      2284                             Bandit Queen  1994
3243      3329                 Year My Voice Broke, The  1987
3669      3759                       Fun and Fancy Free  1947
3679      3769                Thunderbolt and Lightfoot  1974
3685      3775                          Make Mine Music  1946
4978      5073  Son's Room, The (Stanza del figlio, La)  2001
6563      6672                         War Photographer  2001
6667      6776        Lagaan: Once Upon a Time in India  2001
9064     26801      Dragon Inn (Sun lung moon hak chan)  1992
18106    90531                                    Shame  2011

10 Film di atas adalah film-film yang direkomendasikan berdasarkan algoritma Collaborative Filtering, salah satunya adalah Bandit Queen (1994).